# Library

In [2]:
import pandas as pd
import numpy as np
import datetime
from cryptocmd import CmcScraper

import pandas as pd
import numpy as np

from sklearn.model_selection import TimeSeriesSplit

# metrics
from sklearn.metrics import make_scorer, mean_absolute_error, median_absolute_error, mean_squared_error

# cross-validation
from sklearn.model_selection import train_test_split, cross_validate, cross_val_predict, GridSearchCV, RandomizedSearchCV

# preprocessing
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler

# regression models
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [3]:
# Descarga de datos
# ==============================================================================

# Se inicializa el Scraper, se incluye simbolo, inicio y fin de la descarga
scraper = CmcScraper('BTC', '28-04-2013', '28-11-2023')

# Transformar datos recogidos en un dataframe
data = scraper.get_dataframe()
data.sort_values(by='Date', ascending=True, inplace=True)

pd.set_option('display.max_columns', None)
display(data)
pd.reset_option('display.max_columns')

,Date,Open,High,Low,Close,Volume,Market Cap,Time Open,Time High,Time Low,Time Close
3866,2013-04-28,136.689455,143.365942,110.845593,127.954255,2.595684e+04,1.419008e+09,2013-04-28T00:00:00.000Z,2013-04-28T00:45:00.000Z,2013-04-28T13:05:00.000Z,2013-04-28T23:59:59.999Z
3865,2013-04-29,134.444000,147.488007,134.000000,144.539993,0.000000e+00,1.603769e+09,2013-04-29T00:00:00.000Z,2013-04-29T13:15:01.000Z,2013-04-29T05:20:01.000Z,2013-04-29T23:59:59.999Z
3864,2013-04-30,144.000000,146.929993,134.050003,139.000000,0.000000e+00,1.542813e+09,2013-04-30T00:00:00.000Z,2013-04-30T08:25:02.000Z,2013-04-30T18:55:01.000Z,2013-04-30T23:59:59.999Z
3863,2013-05-01,139.000000,139.889999,107.720001,116.989998,0.000000e+00,1.298955e+09,2013-05-01T00:00:00.000Z,2013-05-01T00:15:01.000Z,2013-05-01T19:55:01.000Z,2013-05-01T23:59:59.999Z
3862,2013-05-02,116.379997,125.599998,92.281898,105.209999,0.000000e+00,1.168517e+09,2013-05-02T00:00:00.000Z,2013-05-02T14:25:01.000Z,2013-05-02T14:30:02.000Z,2013-05-02T23:59:59.999Z
...,...,...,...,...,...,...,...,...,...,...,...
4,2023-11-24,37296.316833,38415.338198,37261.606050,37720.281152,2.292296e+10,7.375062e+11,2023-11-24T00:00:00.000Z,2023-11-24T15:41:00.000Z,2023-11-24T00:14:00.000Z,2023-11-24T23:59:59.999Z
3,2023-11-25,37721.413448,37892.429966,37617.416835,37796.792345,9.099571e+09,7.390409e+11,2023-11-25T00:00:00.000Z,2023-11-25T20:05:00.000Z,2023-11-25T11:44:00.000Z,2023-11-25T23:59:59.999Z
2,2023-11-26,37796.828864,37820.301342,37162.749154,37479.122294,1.374480e+10,7.328642e+11,2023-11-26T00:00:00.000Z,2023-11-26T08:35:00.000Z,2023-11-26T17:02:00.000Z,2023-11-26T23:59:59.999Z
1,2023-11-27,37454.191955,37559.354550,36750.127105,37254.168854,1.900293e+10,7.284955e+11,2023-11-27T00:00:00.000Z,2023-11-27T01:08:00.000Z,2023-11-27T16:02:00.000Z,2023-11-27T23:59:59.999Z


In [4]:
import pandas as pd
import requests
import csv

import matplotlib.pyplot as plt
%matplotlib inline
symbol = 'BTCUSDT'
tick_interval = '2h'

'''1m, 3m, 5m, 15m, 30m
1h, 2h, 4h, 6h, 8h, 12h
1d, 3d
1w, 1M'''

'1m, 3m, 5m, 15m, 30m\n1h, 2h, 4h, 6h, 8h, 12h\n1d, 3d\n1w, 1M'

In [5]:
def get_candles(start='', symbol='BTCUSDT', tick_interval='2h', limit=1000):
    
    base_url = 'https://api.binance.com/'
    endpoint = 'api/v3/klines?'
    
    if start:
        query = 'symbol=' + symbol + '&interval=' + tick_interval + '&startTime=' + str(start) +'&limit='+str(limit)
    else:
        query = 'symbol=' + symbol + '&interval=' + tick_interval +'&limit='+str(limit)
        
    candles = requests.get(base_url + endpoint + query).json()
    
    return candles, candles[-1][6]  # return candles and last colse time in a tuple

In [6]:
def get_all_candles_from_start(symbol, tick_interval):  # devuelve una lista de velas, cada vela es una lista tb
    start=1502942400000  # 17 de agosto de 2017
    _, last_time = get_candles(start='', symbol=symbol , tick_interval=tick_interval, limit=1)
    candles = []
    while start < last_time:
        i_candles, next_hop = get_candles(start, symbol, tick_interval)
        candles = candles + i_candles
        start = next_hop
    return candles

In [7]:
candles = get_all_candles_from_start(symbol, tick_interval)

#columns=['open_time','open', 'high', 'low','close','volume','close_time','quote','trades','takers_buy_base','takers_buy_quote','ignore']
#df = pd.DataFrame(candles, columns=columns)

In [8]:
columns=['open_time','open', 'high', 'low','close','volume','close_time','quote','trades','takers_buy_base','takers_buy_quote','ignore']
df = pd.DataFrame(candles, columns=columns)
df

,open_time,open,high,low,close,volume,close_time,quote,trades,takers_buy_base,takers_buy_quote,ignore
0,1502942400000,4261.48000000,4328.69000000,4261.32000000,4315.32000000,70.41592500,1502949599999,302670.96196053,273,56.60857400,243560.75767140,0
1,1502949600000,4330.29000000,4349.99000000,4287.41000000,4349.99000000,11.67294000,1502956799999,50523.37096975,61,7.40515300,32086.66423939,0
2,1502956800000,4333.32000000,4445.78000000,4333.32000000,4444.00000000,11.73643000,1502963999999,51458.85952932,71,11.07305500,48548.12497460,0
3,1502964000000,4441.10000000,4485.39000000,4399.81000000,4427.30000000,51.88345200,1502971199999,231042.29822033,177,47.71457800,212505.92617935,0
4,1502971200000,4436.06000000,4459.00000000,4411.00000000,4459.00000000,54.34483400,1502978399999,241103.15802063,226,41.87703600,185858.30968151,0
...,...,...,...,...,...,...,...,...,...,...,...,...
27497,1701302400000,37854.65000000,37865.21000000,37699.93000000,37703.42000000,1362.48613000,1701309599999,51483850.72127770,66416,620.68220000,23453816.50358820,0
27498,1701309600000,37703.43000000,38051.84000000,37686.84000000,38051.84000000,1726.89081000,1701316799999,65438692.10571030,71301,998.42416000,37831908.91337320,0
27499,1701316800000,38051.84000000,38145.85000000,37853.09000000,37885.63000000,2440.35988000,1701323999999,92740951.78430580,87341,968.68412000,36819305.89848430,0
27500,1701324000000,37885.63000000,37950.00000000,37767.11000000,37792.98000000,2533.50249000,1701331199999,95877478.52288200,84725,905.92460000,34289495.00639330,0


In [9]:
df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')
df['close_time'] = df['close_time'].dt.tz_localize('utc').dt.tz_convert('Europe/Madrid')
df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
df['open_time'] = df['open_time'].dt.tz_localize('utc').dt.tz_convert('Europe/Madrid')

df

,open_time,open,high,low,close,volume,close_time,quote,trades,takers_buy_base,takers_buy_quote,ignore
0,2017-08-17 06:00:00+02:00,4261.48000000,4328.69000000,4261.32000000,4315.32000000,70.41592500,2017-08-17 07:59:59.999000+02:00,302670.96196053,273,56.60857400,243560.75767140,0
1,2017-08-17 08:00:00+02:00,4330.29000000,4349.99000000,4287.41000000,4349.99000000,11.67294000,2017-08-17 09:59:59.999000+02:00,50523.37096975,61,7.40515300,32086.66423939,0
2,2017-08-17 10:00:00+02:00,4333.32000000,4445.78000000,4333.32000000,4444.00000000,11.73643000,2017-08-17 11:59:59.999000+02:00,51458.85952932,71,11.07305500,48548.12497460,0
3,2017-08-17 12:00:00+02:00,4441.10000000,4485.39000000,4399.81000000,4427.30000000,51.88345200,2017-08-17 13:59:59.999000+02:00,231042.29822033,177,47.71457800,212505.92617935,0
4,2017-08-17 14:00:00+02:00,4436.06000000,4459.00000000,4411.00000000,4459.00000000,54.34483400,2017-08-17 15:59:59.999000+02:00,241103.15802063,226,41.87703600,185858.30968151,0
...,...,...,...,...,...,...,...,...,...,...,...,...
27497,2023-11-30 01:00:00+01:00,37854.65000000,37865.21000000,37699.93000000,37703.42000000,1362.48613000,2023-11-30 02:59:59.999000+01:00,51483850.72127770,66416,620.68220000,23453816.50358820,0
27498,2023-11-30 03:00:00+01:00,37703.43000000,38051.84000000,37686.84000000,38051.84000000,1726.89081000,2023-11-30 04:59:59.999000+01:00,65438692.10571030,71301,998.42416000,37831908.91337320,0
27499,2023-11-30 05:00:00+01:00,38051.84000000,38145.85000000,37853.09000000,37885.63000000,2440.35988000,2023-11-30 06:59:59.999000+01:00,92740951.78430580,87341,968.68412000,36819305.89848430,0
27500,2023-11-30 07:00:00+01:00,37885.63000000,37950.00000000,37767.11000000,37792.98000000,2533.50249000,2023-11-30 08:59:59.999000+01:00,95877478.52288200,84725,905.92460000,34289495.00639330,0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27502 entries, 0 to 27501
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype                        
---  ------            --------------  -----                        
 0   open_time         27502 non-null  datetime64[ns, Europe/Madrid]
 1   open              27502 non-null  object                       
 2   high              27502 non-null  object                       
 3   low               27502 non-null  object                       
 4   close             27502 non-null  object                       
 5   volume            27502 non-null  object                       
 6   close_time        27502 non-null  datetime64[ns, Europe/Madrid]
 7   quote             27502 non-null  object                       
 8   trades            27502 non-null  int64                        
 9   takers_buy_base   27502 non-null  object                       
 10  takers_buy_quote  27502 non-null  object                  

In [11]:
df.shape

(27502, 12)

In [16]:
df['low'].value_counts()

8050.00000000     10
10100.00000000     8
6680.00000000      8
6320.00000000      8
11700.00000000     8
                  ..
7323.84000000      1
7365.54000000      1
7345.22000000      1
7342.69000000      1
37553.60000000     1
Name: low, Length: 25022, dtype: int64

In [18]:
df['open'] = df['open'].astype(float)
df['high'] = df['high'].astype(float)
df['low'] = df['low'].astype(float)
df['close'] = df['close'].astype(float)
df['volume'] = df['volume'].astype(float)
df['quote'] = df['quote'].astype(float)
df['trades'] = df['trades'].astype(int)
df['takers_buy_base'] = df['takers_buy_base'].astype(float)
df['takers_buy_quote'] = df['takers_buy_quote'].astype(float)
#df['target'] = df['target'].astype(float)

In [19]:
df.drop(['ignore'], axis=1, inplace=True)

In [ ]:
## prueba de ondas elliott

# Suponiendo que tienes un DataFrame llamado 'crypto_data' con las columnas mencionadas
# Calcula el precio máximo y mínimo en un intervalo de 2 horas
df['max_price'] = df['high'].rolling(window=12).max()
df['min_price'] = df['low'].rolling(window=12).min()

# Calcula el rango de precios
df['price_range'] = df['max_price'] - df['min_price']

# Calcula los niveles de retroceso de Fibonacci
df['fib_61.8'] = df['max_price'] - 0.618 * df['price_range']
df['fib_50'] = df['max_price'] - 0.5 * df['price_range']

# Crea una nueva columna para indicar la presencia de ondas de Elliott y retrocesos de Fibonacci
df['elliott_wave'] = ((df['close'] < df['fib_61.8']) & (df['close'] > df['fib_50'])).astype(int)

In [20]:
df['target'] = df['close'].shift(1).values
df.tail()

,open_time,open,high,low,close,volume,close_time,quote,trades,takers_buy_base,takers_buy_quote,target
27497,2023-11-30 01:00:00+01:00,37854.65,37865.21,37699.93,37703.42,1362.48613,2023-11-30 02:59:59.999000+01:00,5.148385e+07,66416,620.68220,2.345382e+07,37854.64
27498,2023-11-30 03:00:00+01:00,37703.43,38051.84,37686.84,38051.84,1726.89081,2023-11-30 04:59:59.999000+01:00,6.543869e+07,71301,998.42416,3.783191e+07,37703.42
27499,2023-11-30 05:00:00+01:00,38051.84,38145.85,37853.09,37885.63,2440.35988,2023-11-30 06:59:59.999000+01:00,9.274095e+07,87341,968.68412,3.681931e+07,38051.84
27500,2023-11-30 07:00:00+01:00,37885.63,37950.00,37767.11,37792.98,2533.50249,2023-11-30 08:59:59.999000+01:00,9.587748e+07,84725,905.92460,3.428950e+07,37885.63
27501,2023-11-30 09:00:00+01:00,37792.98,37814.48,37553.60,37701.10,2424.16748,2023-11-30 10:59:59.999000+01:00,9.133959e+07,94800,1047.63700,3.947819e+07,37792.98


In [21]:
df = df.dropna()
df.shape

(27501, 12)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27501 entries, 1 to 27501
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype                        
---  ------            --------------  -----                        
 0   open_time         27501 non-null  datetime64[ns, Europe/Madrid]
 1   open              27501 non-null  float64                      
 2   high              27501 non-null  float64                      
 3   low               27501 non-null  float64                      
 4   close             27501 non-null  float64                      
 5   volume            27501 non-null  float64                      
 6   close_time        27501 non-null  datetime64[ns, Europe/Madrid]
 7   quote             27501 non-null  float64                      
 8   trades            27501 non-null  int32                        
 9   takers_buy_base   27501 non-null  float64                      
 10  takers_buy_quote  27501 non-null  float64                 

In [29]:
df2 = df.drop(['open_time', 'close_time'], axis=1)
df2

,open,high,low,close,volume,quote,trades,takers_buy_base,takers_buy_quote,target
1,4330.29,4349.99,4287.41,4349.99,11.672940,5.052337e+04,61,7.405153,3.208666e+04,4315.32
2,4333.32,4445.78,4333.32,4444.00,11.736430,5.145886e+04,71,11.073055,4.854812e+04,4349.99
3,4441.10,4485.39,4399.81,4427.30,51.883452,2.310423e+05,177,47.714578,2.125059e+05,4444.00
4,4436.06,4459.00,4411.00,4459.00,54.344834,2.411032e+05,226,41.877036,1.858583e+05,4427.30
5,4459.00,4485.39,4333.42,4352.34,120.217167,5.331356e+05,632,83.307097,3.695614e+05,4459.00
...,...,...,...,...,...,...,...,...,...,...
27497,37854.65,37865.21,37699.93,37703.42,1362.486130,5.148385e+07,66416,620.682200,2.345382e+07,37854.64
27498,37703.43,38051.84,37686.84,38051.84,1726.890810,6.543869e+07,71301,998.424160,3.783191e+07,37703.42
27499,38051.84,38145.85,37853.09,37885.63,2440.359880,9.274095e+07,87341,968.684120,3.681931e+07,38051.84
27500,37885.63,37950.00,37767.11,37792.98,2533.502490,9.587748e+07,84725,905.924600,3.428950e+07,37885.63


In [30]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27501 entries, 1 to 27501
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   open              27501 non-null  float64
 1   high              27501 non-null  float64
 2   low               27501 non-null  float64
 3   close             27501 non-null  float64
 4   volume            27501 non-null  float64
 5   quote             27501 non-null  float64
 6   trades            27501 non-null  int32  
 7   takers_buy_base   27501 non-null  float64
 8   takers_buy_quote  27501 non-null  float64
 9   target            27501 non-null  float64
dtypes: float64(9), int32(1)
memory usage: 2.2 MB


In [33]:
# Make sure all columns are numeric
assert df.drop(['open_time', 'close_time'], axis=1).select_dtypes(include='number').shape[1] == df.drop(['open_time', 'close_time'], axis=1).shape[1]

In [48]:
scoring = {
    'MAE': make_scorer(mean_absolute_error),
    'MedAE': make_scorer(median_absolute_error),
    'MSE': make_scorer(mean_squared_error)
}

In [62]:
def custom_cv_ts(df, models, X, y, cv):
    """
    Perform time series cross-validation with a given model, X, y, and cv object.
    """
    # Create empty train and test MAE lists
    train_maes = []
    val_maes = []
    train_mse = []
    val_mse= []
    train_medae = []
    val_medae = []
    best_pa = []

    # Cross-validation
    for i, (train_index, val_index) in enumerate(cv.split(X)):
        # Train and val data in this split
        X_train, y_train = X.iloc[train_index], y.iloc[train_index]
        X_val, y_val = X.iloc[val_index], y.iloc[val_index]

        # Nested train and validation data from previous train data
        X_train2, X_val2, y_train2, y_val2 = train_test_split(X_train, y_train, test_size=0.1,
                                                              random_state=42, shuffle=False)
        
        # Calculate validation metrics after training each model
        metrics = []
        for dtree in models:
            dtree.fit(X_train2, y_train2)
            y_pred_val2 = dtree.predict(X_val2)
            metrics.append( mean_absolute_error(y_val2, y_pred_val2) )
            
        # Select the best model based on validation MAE and retrain it with the whole training data
        best_model_idx = np.argmin(metrics)
        best_model = models[best_model_idx]
        best_model.fit(X_train, y_train)
        
        # Predict and compute MAEs of the train and val sets
        y_pred_train = best_model.predict(X_train)
        y_pred_val = best_model.predict(X_val)
        train_maes.append( mean_absolute_error(y_train, y_pred_train) )
        val_maes.append( mean_absolute_error(y_val, y_pred_val) )
        train_mse.append( mean_squared_error(y_train, y_pred_train) )
        val_mse.append( mean_squared_error(y_val, y_pred_val) )
        train_medae.append( median_absolute_error(y_train, y_pred_train) )
        val_medae.append( median_absolute_error(y_val, y_pred_val) )
        if isinstance(best_model, RandomizedSearchCV):        
            best_pa.append(best_model.best_params_)
                                    
        # Show info
        print(i)
        dates = df.loc[X.index, 'open_time']
        dates2 = df.loc[X.index, 'close_time']
        print('Best validation model:', best_model_idx)
        print('Train dates:', dates.iloc[train_index].min().date(), '-', dates.iloc[train_index].max().date())
        print('Train MAE:', round(train_maes[-1], 2))
        print('Train MSE:', round(train_mse[-1], 2))
        print('Train MedAE:', round(train_medae[-1], 2))
        print('Validation dates:', dates.iloc[val_index].min().date(), '-', dates.iloc[val_index].max().date())
        print('Validation MAE:', round(val_maes[-1], 2))
        print('Validation MSE:', round(val_mse[-1], 2))
        print('Validation MedAE:', round(val_medae[-1], 2))
        print()
        if isinstance(best_model, RandomizedSearchCV):  
            print("Best Parameters: ",best_model.best_params_)

    return np.array(train_maes), np.array(val_maes), np.array(train_mse), np.array(val_mse), np.array(train_medae), np.array(val_medae), np.array(best_pa)

In [50]:
X = df.drop(['open_time', 'close_time', 'target'], axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [51]:
X

,open,high,low,close,volume,quote,trades,takers_buy_base,takers_buy_quote,ignore
2,4333.32000000,4445.78000000,4333.32000000,4444.00000000,11.73643000,51458.85952932,71,11.07305500,48548.12497460,0
3,4441.10000000,4485.39000000,4399.81000000,4427.30000000,51.88345200,231042.29822033,177,47.71457800,212505.92617935,0
4,4436.06000000,4459.00000000,4411.00000000,4459.00000000,54.34483400,241103.15802063,226,41.87703600,185858.30968151,0
5,4459.00000000,4485.39000000,4333.42000000,4352.34000000,120.21716700,533135.64029663,632,83.30709700,369561.44837959,0
6,4352.33000000,4354.84000000,4235.45000000,4289.24000000,127.12370300,547509.80307280,590,77.99427100,336682.03719969,0
...,...,...,...,...,...,...,...,...,...,...
27487,37940.49000000,38040.03000000,37871.10000000,38035.73000000,1216.02327000,46126656.36434700,44765,539.64809000,20473815.37742260,0
27488,38035.72000000,38235.36000000,37982.20000000,38096.69000000,2585.99565000,98534339.68041920,76491,1279.48533000,48749838.05111640,0
27489,38096.70000000,38450.00000000,38075.00000000,38281.00000000,4373.54133000,167260587.59773450,133916,2329.37420000,89099284.23144130,0
27490,38280.99000000,38292.71000000,37878.17000000,38104.65000000,3799.14439000,144656959.91782310,123811,1691.06095000,64391649.22672210,0


In [63]:
# Create TimeSeriesSplit object
tscv = TimeSeriesSplit(n_splits=50, test_size=3)

# Models to try in each cross-validation split
models = [
    DecisionTreeRegressor(max_depth=2),
    DecisionTreeRegressor(max_depth=4),
    DecisionTreeRegressor(max_depth=6),
]

train_maes, valid_maes, train_mse, valid_mse, train_medae, valid_medae, best_pa = custom_cv_ts(df, models, X_train, y_train, cv=tscv)

0
Best validation model: 2
Train dates: 2017-08-17 - 2022-08-15
Train MAE: 190.39
Train MSE: 60503.25
Train MedAE: 157.25
Validation dates: 2022-08-16 - 2022-08-16
Validation MAE: 79.99
Validation MSE: 7676.66
Validation MedAE: 61.76

1
Best validation model: 2
Train dates: 2017-08-17 - 2022-08-16
Train MAE: 190.41
Train MSE: 60534.01
Train MedAE: 157.25
Validation dates: 2022-08-16 - 2022-08-16
Validation MAE: 122.22
Validation MSE: 18575.59
Validation MedAE: 88.07

2
Best validation model: 2
Train dates: 2017-08-17 - 2022-08-16
Train MAE: 190.38
Train MSE: 60521.36
Train MedAE: 157.27
Validation dates: 2022-08-16 - 2022-08-16
Validation MAE: 146.86
Validation MSE: 27866.53
Validation MedAE: 114.61

3
Best validation model: 2
Train dates: 2017-08-17 - 2022-08-16
Train MAE: 190.28
Train MSE: 60491.19
Train MedAE: 157.15
Validation dates: 2022-08-16 - 2022-08-16
Validation MAE: 250.33
Validation MSE: 63333.32
Validation MedAE: 248.58

4
Best validation model: 2
Train dates: 2017-08-17 -

In [64]:
print('Mean train MAE:', train_maes.mean().round(2), '$')
print('Mean validation MAE:', valid_maes.mean().round(2), '$')
print()
print('Mean train MSE:', train_mse.mean().round(2), '$')
print('Mean validation MSE:', valid_mse.mean().round(2), '$')
print()
print('Mean train MedAE:', train_medae.mean().round(2), '$')
print('Mean validation MedAE:', valid_medae.mean().round(2), '$')

Mean train MAE: 190.14 $
Mean validation MAE: 183.36 $

Mean train MSE: 60407.98 $
Mean validation MSE: 44430.95 $

Mean train MedAE: 158.15 $
Mean validation MedAE: 180.42 $


In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
diff_open_close = df2['close'] - df2['open']
buy_sell = pd.Series(np.zeros(len(df2['close'])))
buy_sell.iloc[np.where(diff_open_close > 0)[0]] = 1
buy_sell


In [ ]:
y_real_bin2.iloc[np.where(comp_vend > 0)[0]] = 1
y_real_bin2

In [ ]:
y_diff = y_train - X_train.close
y_diff
y_real_bin = pd.Series(np.zeros(len(y_train)))
y_real_bin.iloc[np.where(y_diff > 0)[0]] = 1
y_real_bin
